In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 필터 커널 세팅
kernel = np.ones((3, 3), np.uint8)
kernel2 = np.ones((5, 5), np.uint8)

# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 240])

red3 = np.array([155, 50, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])

In [46]:
img = cv2.imread('./sample6_68.jpg', 0)

img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
# img1 = img.copy()
cv2.imshow('img', img)

contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

img_external = np.zeros(img.shape, img.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(img_external, contours, i, 255, -1)

img_internal = np.zeros(img.shape, img.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] != -1 :
        cv2.drawContours(img_internal, contours, i, 255, -1)
        
cv2.imshow('img_external', img_external)
cv2.imshow('img_internal', img_internal)
cv2.waitKey()
cv2.destroyAllWindows()        

In [42]:
# 필터 커널 세팅
kernel = np.ones((3, 3), np.uint8)
kernel2 = np.ones((5, 5), np.uint8)

# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 240])

red3 = np.array([155, 50, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])


img = cv2.imread('./sample6_68.jpg')
img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
# img = cv2.GaussianBlur( img, (3,3), 0 )
img1 = img.copy()

blur = cv2.GaussianBlur( img, (3,3), 0 )    
# HSV 영상으로 변환  
hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 

# 색상에 따른 마스크 생성
mask_red1 = cv2.inRange(hsv, red1, red2)
mask_red2 = cv2.inRange(hsv, red3, red4)

# 마스크 결합
mask_red = mask_red1 + mask_red2

# 비트 and 연산
res_red2 = cv2.bitwise_and(img, img, mask = mask_red)

# 침식 (노이즈제거) -> 팽창(픽셀 살리기) 
erosion_image = cv2.erode(mask_red, kernel, iterations=1)  
dilation_image = cv2.dilate(erosion_image, kernel, iterations=1) 
img = dilation_image
# cv2.imshow('erosion_image', erosion_image)
# cv2.imshow('dilation_image', dilation_image)
contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
# print(contours)
# print(hierarchy)

img_external = np.zeros(img.shape, img.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(img_external, contours, i, 255, -1)
erosion_img_external = cv2.erode(img_external, kernel, iterations=1)  
dilation_img_external = cv2.dilate(erosion_img_external, kernel2, iterations=1) 
e = cv2.moments(dilation_img_external)
# print(e['m10'] / e['m00'])
# print(e['m01'] / e['m00'])        
# cv2.circle(img_external, ( int(e['m10'] / e['m00']), int(e['m01'] / e['m00']) ), 2, (0, 255, 0), 2)

##########################################################################################
img_internal = np.zeros(img.shape, img.dtype)
cv2.imshow('img_internal1', img_internal)
for i in range(len(contours)) :
    if hierarchy[0][i][3] != -1 :
        cv2.drawContours(img_internal, contours, i, 255, -1)
cv2.imshow('img_internal2', img_internal)

erosion_img_internal = cv2.erode(img_internal, kernel, iterations=1)
cv2.imshow('erosion_img_internal', erosion_img_internal)


dilation_img_internal = cv2.dilate(erosion_img_internal, kernel2, iterations=1) 
cv2.imshow('dilation_img_internal', dilation_img_internal)

i = cv2.moments(dilation_img_internal)        
# print(i['m10'] / i['m00'])
# print(i['m01'] / i['m00'])    
cv2.circle(img_internal, ( int(i['m10'] / i['m00']), int(i['m01'] / i['m00']) ), 2, (0, 255, 0), 2)


########################################  객체 분리  ######################################
numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(dilation_img_internal)

for idx, centroid in enumerate(centroids) :
#     if stats[idx][0] == 0 and stats[idx][1] == 0 :
#         continue
#     if np.any(np.isnan(centroid)) :
#         continue

    x, y, w, h, area = stats[idx]
    centerX, centerY = int(centroid[0]), int(centroid[1])
    # cv2.rectangle(img1, (x-5, y-5), (x+w+5, y+h+5), (255, 0, 0), 1)
    # cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
    if y < 500 and area < 4500 and area > 80 and abs(w-h) < 5 : # if area > 120 and abs(w-h) < 5 : # abs : 정사각형에 가까운 네모
        print(x, y, w, h, area)
        cv2.rectangle(img1, (x-5, y-5), (x+w+5, y+h+5), (255, 0, 0), 1)

        cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
        # 원 검출 영역
        detected_img = img[y : y+h,  x : x+w]
        # cv2.imshow('detected_img', detected_img)

        # gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)                 
        # 원 검출
        circles = cv2.HoughCircles(detected_img,
                                   cv2.HOUGH_GRADIENT,
                                   1, 
                                   100,
                                   param1=200,
                                   param2=30, 
                                   minRadius=5, 
                                   maxRadius=40)
        print(circles)

        if circles is not None :
            circles = np.uint16(np.around(circles))

            for i in circles[0, :] :
                cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
                cv2.rectangle(img1, (x-5, y-5), (x+w+5, y+h+5), (0, 255, 0), 1)

# cv2.circle(img1, ( int(e['m10'] / e['m00']), int(e['m01'] / e['m00']) ), 2, (0, 255, 0), 2)
# cv2.imshow('img', img)
cv2.imshow('img1', img1)
cv2.imshow('img_external', img_external)
# cv2.imshow('img_internal', img_internal)
cv2.waitKey()
cv2.destroyAllWindows()

### 이미지 모멘트 계산

In [ ]:
# 이진 및 그레이스케일 이미지에서 모멘트를 계산하는 방법

In [33]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [34]:
image = np.zeros((480, 640), np.uint8)
cv2.ellipse(image, (320, 240), (200, 100), 0, 0, 360, 255, -1)

cv2.imshow('image', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [35]:
m = cv2.moments(image)
for name, val in m.items() :
    print(name, '/t', val)

m00 /t 16119315.0
m10 /t 5158101240.0
m01 /t 3868620810.0
m20 /t 1812142855350.0
m11 /t 1237939564800.0
m02 /t 969157708320.0
m30 /t 683285449618080.0
m21 /t 434912202354750.0
m12 /t 310125260718570.0
m03 /t 252129278267070.0
mu20 /t 161575917357.3161
mu11 /t -72.99898365715876
mu02 /t 40692263506.429634
mu30 /t 1687957749.1273296
mu21 /t -420182048.74010897
mu12 /t -422443285.191627
mu03 /t 105694127.70601428
nu20 /t 0.0006218468887998856
nu11 /t -2.809465148965929e-13
nu02 /t 0.00015660970937729057
nu30 /t 1.6180618413372907e-09
nu21 /t -4.027829131581005e-10
nu12 /t -4.049505150535472e-10
nu03 /t 1.0131748557269813e-10


In [37]:
print(m['m10'] / m['m00'])
print(m['m01'] / m['m00'])

319.9950643063927
239.999082467214
